In [47]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import json
from typing import List, Dict, Any
import requests
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.tools import BaseTool
from pydantic import BaseModel, Field
from pathlib import Path
import http.client

Create Integrated plan using info of:
- Product Name and Type
- Product Price and Materials 
- Actual Date
- Product actual CurrentStock
- Product Pridected CurrentStock

In [19]:
load_dotenv()

DEEPSEEK_API_KEY = os.getenv('DEEPSEEK_API_KEY')

In [20]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=DEEPSEEK_API_KEY, 
)

In [4]:
completion = client.chat.completions.create(
  extra_body={},
  model="deepseek/deepseek-r1-distill-llama-70b:free",
  messages=[
    {
      "role": "user",
      "content": "What is the meaning of life?"
    }
  ]
)

In [5]:
print(completion.choices[0].message.content)

The question of the meaning of life is one of the most profound and enduring questions humans have grappled with throughout history. Different people, cultures, religions, and philosophies have offered a wide range of answers. Here are a few perspectives:

1. **Philosophical Perspectives:**
   - **Aristotle:** Suggested that the meaning of life is to achieve happiness and fulfillment through living a virtuous life and cultivating reason.
   - **Existentialism:** Argues that life has no inherent meaning, and it is up to each individual to create their own meaning.
   - **Nihilism:** Proposes that life lacks intrinsic meaning, purpose, or value.

2. **Religious and Spiritual Perspectives:**
   - **Abrahamic Religions (Judaism, Christianity, Islam):** Often suggest that the meaning of life is to know, love, and serve a divine being or to fulfill a divine plan.
   - **Buddhism:** Teaches that the purpose of life is to attain enlightenment and escape the cycle of suffering and rebirth.
   -

In [2]:
import random
from datetime import datetime, timedelta
import json
import uuid

class DummyProductDataGenerator:
    PRODUCT_TYPES = [
        "Smart Home Device", "IoT Sensor", "Wearable Technology", 
        "Electronic Accessory", "Home Appliance"
    ]
    
    MATERIALS = [
        "Aluminum", "Plastic", "Recycled Polymer", "Carbon Fiber", 
        "Stainless Steel", "Glass", "Silicon"
    ]

    @classmethod
    def generate_product(cls, name=None):
        """Generate a comprehensive dummy product dataset"""
        current_date = datetime.now()
        
        return {
            "id": str(uuid.uuid4()),
            "name": name or f"{random.choice(['Smart', 'Advanced', 'Pro'])} {random.choice(['Home', 'Tech', 'Connect'])} {random.choice(['Device', 'Sensor', 'System'])}",
            "type": random.choice(cls.PRODUCT_TYPES),
            "price": round(random.uniform(50, 500), 2),
            "materials": random.sample(cls.MATERIALS, k=random.randint(1, 3)),
            "current_date": current_date.isoformat(),
            "current_stock": random.randint(100, 5000),
            "predicted_current_stock": random.randint(50, 5000),
            "stock_trend": random.choice(["Increasing", "Stable", "Decreasing"]),
            "manufacturing_details": {
                "batch_number": f"BATCH-{random.randint(1000, 9999)}",
                "production_date": (current_date - timedelta(days=random.randint(1, 60))).isoformat()
            }
        }

    @classmethod
    def generate_product_catalog(cls, num_products=10):
        """Generate a catalog of dummy products"""
        return [cls.generate_product() for _ in range(num_products)]

def save_dummy_data(filename='product_catalog.json'):
    """Save generated dummy data to a JSON file"""
    catalog = DummyProductDataGenerator.generate_product_catalog()
    with open(filename, 'w') as f:
        json.dump(catalog, f, indent=2)
    return catalog

def load_dummy_data(filename='product_catalog.json'):
    """Load dummy data from JSON file"""
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return save_dummy_data(filename)

# Example usage
if __name__ == "__main__":
    # Generate and save dummy data
    products = save_dummy_data()
    print(json.dumps(products, indent=2))

[
  {
    "id": "2290cbd3-fd62-463f-92ed-f5f4a9dae7ea",
    "name": "Smart Tech System",
    "type": "Wearable Technology",
    "price": 325.5,
    "materials": [
      "Silicon",
      "Plastic",
      "Recycled Polymer"
    ],
    "current_date": "2025-02-07T18:33:27.666408",
    "current_stock": 2729,
    "predicted_current_stock": 3781,
    "stock_trend": "Stable",
    "manufacturing_details": {
      "batch_number": "BATCH-7520",
      "production_date": "2024-12-28T18:33:27.666408"
    }
  },
  {
    "id": "afbb792b-48e4-47ad-ae4a-e805793c9ca3",
    "name": "Advanced Tech Sensor",
    "type": "Electronic Accessory",
    "price": 438.79,
    "materials": [
      "Aluminum",
      "Plastic",
      "Glass"
    ],
    "current_date": "2025-02-07T18:33:27.668410",
    "current_stock": 2572,
    "predicted_current_stock": 2118,
    "stock_trend": "Decreasing",
    "manufacturing_details": {
      "batch_number": "BATCH-7353",
      "production_date": "2024-12-30T18:33:27.668410"
    }


In [53]:
class ProductPlanningState(BaseModel):
    product_info: Dict[str, Any] = Field(default_factory=dict)
    integrated_plan: str = ""
    web_search_results: Dict[str, Any] = {}

class WebSearchTool(BaseTool):
    name: str = "web_search_tool"
    description: str = "Search the web for additional product information"

    def _run(self, query: str) -> str:
        try:
            # Replace with your actual web search API endpoint
            conn = http.client.HTTPSConnection("google.serper.dev")
            payload = json.dumps({
                "q": query
            })
            headers = {
                'X-API-KEY': '9e47c410e6f594092fd799e42873d35a38de67d4',
                'Content-Type': 'application/json'
            }
            conn.request("POST", "/search", payload, headers)
            res = conn.getresponse()
            data_bytes = res.read()
            data_str = data_bytes.decode("utf-8")
            # Correctly parse the JSON string into a Python dictionary
            data = json.loads(data_str)
            return data
        except Exception as e:
            return f"Error in web search: {str(e)}"

class ProductEndpointTool(BaseTool):
    name: str = "product_endpoint_tool"
    description: str = "Fetch product information from endpoint"
    json_file: Path = Field(default=Path("product_catalog.json"))

    # def _run(self, product_name: str) -> Dict[str, Any]:
    #     try:
    #         # Replace with your actual product endpoint
    #         # response = requests.get(f"https://your-product-endpoint.com/product?name={product_name}")
    #         response = json.load()
    #         return response.json()
    #     except Exception as e:
    #         return f"Error fetching product info: {str(e)}"
        
    def _run(self, product_name: str) -> Dict[str, Any]:
        try:
            # Load JSON from file
            with open(self.json_file, 'r') as f:
                products = json.load(f)
            
            # Filter products by name (exact match)
            matches = [p for p in products if p["name"] == product_name]
            
            return {
                "count": len(matches),
                "results": matches
            }
            
        except FileNotFoundError:
            return {"error": f"File {self.json_file} not found"}
        except json.JSONDecodeError:
            return {"error": f"Invalid JSON format in {self.json_file}"}
        except Exception as e:
            return {"error": f"Unexpected error: {str(e)}"}

In [60]:
def fetch_product_data(state: ProductPlanningState, tool: ProductEndpointTool) -> ProductPlanningState:
    """Fetch initial product data from endpoint"""
    product_name = state.product_info.get('name', '') # change name
    product_info = tool._run(product_name)
    state.product_info.update(product_info)
    return state

def web_search_additional_info(state: ProductPlanningState, tool: WebSearchTool) -> ProductPlanningState:
    """Perform web search to gather additional product insights"""
    query = f"Current market trends for {state.product_info.get('name', '')} product" # change name
    search_results = tool._run(query)
    state.web_search_results = search_results
    return state

def generate_integrated_plan(state: ProductPlanningState, llm: ChatOpenAI) -> ProductPlanningState:
    """Generate comprehensive product planning using collected information"""
    # Combine product endpoint data and web search results
    context = {
        "product_info": state.product_info,
        "web_insights": state.web_search_results
    }
    
    # Generate integrated plan prompt
    prompt = f"""Create a comprehensive product planning report based on the following information:

Product Details:
- Name: {context['product_info'].get('name', 'N/A')}
- Type: {context['product_info'].get('type', 'N/A')}
- Current Price: {context['product_info'].get('price', 'N/A')}
- Materials: {context['product_info'].get('materials', 'N/A')}
- Current Stock: {context['product_info'].get('current_stock', 'N/A')}
- Predicted Stock: {context['product_info'].get('predicted_stock', 'N/A')}

Additional Web Insights: {context['web_insights']}

Integrated Product Plan:
"""
    
    # Generate plan using LLM
    response = llm.invoke(prompt)
    state.integrated_plan = response.content
    return state

In [61]:
def build_product_planning_workflow():
    """Construct the LangGraph workflow for product planning"""
    workflow = StateGraph(ProductPlanningState)
    
    # Define tools
    product_tool = ProductEndpointTool()
    web_search_tool = WebSearchTool()
    
    llm = ChatOpenAI(
        model="deepseek/deepseek-r1-distill-llama-70b:free",  # use "gpt-4-turbo" if that's what you prefer
        openai_api_key=DEEPSEEK_API_KEY,
        openai_api_base="https://openrouter.ai/api/v1"
    )

    
    # Add nodes
    workflow.add_node("fetch_product_data", 
                      lambda state: fetch_product_data(state, product_tool))
    workflow.add_node("web_search", 
                      lambda state: web_search_additional_info(state, web_search_tool))
    workflow.add_node("generate_plan", 
                      lambda state: generate_integrated_plan(state, llm))
    
    # Define edges
    workflow.set_entry_point("fetch_product_data")
    workflow.add_edge("fetch_product_data", "web_search")
    workflow.add_edge("web_search", "generate_plan")
    workflow.add_edge("generate_plan", END)
    
    # Compile the graph
    return workflow.compile()

# Example usage
def main():
    # Initialize workflow
    workflow = build_product_planning_workflow()
    
    # Initial product info
    initial_state = ProductPlanningState(
        product_info={
            "name": "Smart Home Sensor",
            "type": "Wearable Technology"
        }
    )
    
    # Execute workflow
    result = workflow.invoke(initial_state)
    print(result["integrated_plan"])
    return result



In [62]:
if __name__ == "__main__":
    result = main()

**Comprehensive Product Planning Report: Smart Home Sensor (Wearable Technology)**

---

**Executive Summary:**
The Smart Home Sensor is an innovative wearable device designed to integrate smart home technology with personal wearable tech. Targeting homeowners and tech enthusiasts, it offers a unique solution for seamless smart home control on the go. Leveraging the growth of both smart home and wearable markets, this product aims to fill a niche by providing a wearable sensor that integrates with existing smart home ecosystems.

---

**Market Analysis:**
- **Target Market:** Homeowners, tech-savvy individuals, and early adopters seeking integrated smart home solutions.
- **Market Trends:** The global smart home market is projected to reach $250.6 billion by 2029 (CAGR 9.55% from 2025-2029). The wearable tech market is also growing, presenting a synergistic opportunity.
- **Growth Potential:** Rapid adoption in North America, with smart home penetration expected to reach 92.5% by 2029.

In [70]:
result['product_info']

{'name': 'Smart Home Sensor',
 'type': 'Wearable Technology',
 'count': 1,
 'results': [{'id': '55ca8ab3-9622-475a-86b4-fe5210afe2af',
   'name': 'Smart Home Sensor',
   'type': 'Electronic Accessory',
   'price': 143.7,
   'materials': ['Silicon'],
   'current_date': '2025-02-07T18:33:27.668410',
   'current_stock': 4905,
   'predicted_current_stock': 4657,
   'stock_trend': 'Increasing',
   'manufacturing_details': {'batch_number': 'BATCH-5007',
    'production_date': '2024-12-12T18:33:27.668410'}}]}